In [1]:
import argparse
import inspect
import logging
import math
import os
from pathlib import Path
from typing import Optional

import torch
import torch.nn.functional as F

import datasets
import diffusers
from accelerate import Accelerator
from accelerate.logging import get_logger
from datasets import load_dataset
from diffusers import DDPMPipeline, DDPMScheduler, UNet2DModel
from diffusers.optimization import get_scheduler
from diffusers.training_utils import EMAModel
from diffusers.utils import check_min_version
from huggingface_hub import HfFolder, Repository, create_repo, whoami
from torchvision.transforms import (
    CenterCrop,
    Compose,
    InterpolationMode,
    Normalize,
    RandomHorizontalFlip,
    Resize,
    ToTensor,
)
from tqdm.auto import tqdm
import yaml

In [4]:
# Model hyperparameters

in_channels=3,  # the number of input channels, 3 for RGB images
out_channels=3,  # the number of output channels
layers_per_block=2,  # how many ResNet layers to use per UNet block
block_out_channels=(128, 128, 256, 256, 512, 512),  # More channels -> more parameters
down_block_types=(
    "DownBlock2D",  # a regular ResNet downsampling block
    "DownBlock2D",
    "DownBlock2D",
    "DownBlock2D",
    "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
    "AttnDownBlock2D",
),

# create config dictionary

config = {
    "in_channels": 3, # the number of input channels, 3 for RGB images
    "out_channels": 3, # the number of output channels
    "layers_per_block": 2, # how many ResNet layers to use per UNet block
    "block_out_channels": (128, 128, 256, 256, 512, 512), # More channels -> more parameters
    "down_block_types": (
    "DownBlock2D",  # a regular ResNet downsampling block
    "DownBlock2D",
    "DownBlock2D",
    "DownBlock2D",
    "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
    "AttnDownBlock2D",
),       
}

# save config to yaml file

with open('config.yaml', 'w') as file:
    documents = yaml.dump(config, file)

In [19]:
#open config yaml config file
with open('config.yaml', 'r') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
print(f'The keys are: {config.keys()}')
config['down_block_types']

The keys are: dict_keys(['block_out_channels', 'down_block_types', 'in_channels', 'layers_per_block', 'out_channels'])


('DownBlock2D',
 'DownBlock2D',
 'DownBlock2D',
 'DownBlock2D',
 'AttnDownBlock2D',
 'AttnDownBlock2D')